In [2]:
import snowflake.connector
import os
import pyodbc
import pandas as pd
import datetime

account = 'ffa72186.us-east-1'

user = 'B042304@MERCANTIL.COM.BR'
password = 'trabalho@03T'

warehouse = 'WH_DEV'
database = 'SDX_GERENCIA_CRESCIMENTO'
schema = 'CONHECIMENTO_CLIENTE_BRONZE'
authenticator = 'externalbrowser'

conn_snow = snowflake.connector.connect(
    user=user,
    authenticator=authenticator,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema
)

cur_snow = conn_snow.cursor()

# Conexão SQLGDNCLI
server = 'SQLGDNCLI'
database = 'WORKSPACE'
driver = '{ODBC Driver 17 for SQL Server}'

conn_cli = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server +
                      ';DATABASE=' + database + ';Trusted_Connection=yes')
# Cursor para executar comandos SQL
cur_cli = conn_cli.cursor()

# Conexão SQLGDNP
server = 'SQLGDNP'
database = 'SHARE'
driver = '{ODBC Driver 17 for SQL Server}'

conn_p = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server +
                      ';DATABASE=' + database + ';Trusted_Connection=yes')
# Cursor para executar comandos SQL
cur_p = conn_p.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/544ce133-0d39-4cbb-940b-1f0f28a88b59/saml2?SAMLRequest=lZJRb9sgGEX%2FisWebQNxMgfFqdxGXTO1W5o41bY3jHGKakPKh%2Bvm3484jdQ9tNLeLHwuHLjf7OK1bYIXaUEZnSESYRRILUyl9C5D2%2BI6TFEAjuuKN0bLDB0koIv5DHjb7FneuUe9ls%2BdBBf4jTSw4UeGOquZ4aCAad5KYE6wTX53y2iE2d4aZ4Rp0LvI5wkOIK3zhudIBcrrPTq3Z3Hc933UjyJjdzHFGMd4GnvqiHw586%2F%2BTh%2FwJMbJkfeEx1dvbpdKn57gM63yBAG7KYpVuPq5KVCQn1WvjIaulXYj7YsScru%2BPQmAN6hr%2FpWSdBJ1EEoOLiQRaNPXDX%2BSwrT7zvltI%2F8V17KKG7NT%2FubLRYb2T6q6ye8PD9t7%2BLaWdoefl0ZsRFpPfo2t%2BV18tzy%2FzCsqeF7eYYGCh3O19FjtEqCTS30s1PklTJMQT0KaFIQwShhOopQmf1Cw8IUqzd2QPFsPHlGrhDVgamd0o7QcLMdJIiQZjUJcjaZhIsoynCa4DEmNa5ryNC3H0%2FhYM0Wn0WGDiJ3%2F94PM4vfxtzH84ZtZLlamUeIQXBvbcvdxcSQiw4qqwnpAmWy5avKqshLAF9g0pr%2Bykjs%2F7c52EsXz06n%2Fzv

In [3]:
ano = 2020
mes = 1

while(ano == 2020):
    query = f"""
        select DTA_RFC, NUM_PES, COD_PDT, NUM_BNF_INS, VLR_LPA, VLR_LPA_DPN_ONL, VLR_LPA_DPN_CNL
        from SHARE.[MERCANTIL\B031684].[TB_TC_LPA_INSS_HST_2020]
        where year(dta_rfc) = {ano}
        and month(dta_rfc) = {mes}
    """

    df = pd.read_sql_query(query, conn_p, chunksize=200000)

    cur_snow.fast_executemany = True

    i = 1

    for chunk in df:

        # Substituir valores NaN por 0 no DataFrame
        chunk = chunk.fillna(0)
        
        # Converter colunas para string
        chunk = chunk.astype(str)

        # Preparar os dados para inserção em formato de lista de tuplas
        data_to_insert = [tuple(row) for row in chunk.values]

        # Inserir os dados na tabela do Snowflake
        insert_query = """INSERT INTO SDX_GERENCIA_CRESCIMENTO.CONHECIMENTO_CLIENTE_SILVER.B042304_TC_LPA_INSS
        (DTA_RFC, NUM_PES, NUM_BNF_INS, COD_PDT, VLR_LPA, VLR_LPA_DPN_CNL, VLR_LPA_DPN_ONL)
        VALUES (%s,%s,%s,%s,%s,%s,%s)"""

        cur_snow.executemany(insert_query, data_to_insert)
        conn_snow.commit()

        i += 1

    conn_snow.commit()
    
    print(ano, ' ', mes, ' - ', datetime.datetime.now())

    if(mes == 12):
        mes = 1
        ano += 1
    else:
        mes += 1

C:\Users\B042304\AppData\Local\Temp\ipykernel_17372\1331809403.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn_p, chunksize=200000)


2020   1  -  2024-06-24 14:00:25.350350


C:\Users\B042304\AppData\Local\Temp\ipykernel_17372\1331809403.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn_p, chunksize=200000)


2020   2  -  2024-06-24 17:10:23.824564


C:\Users\B042304\AppData\Local\Temp\ipykernel_17372\1331809403.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn_p, chunksize=200000)


...

2020   10  -  2024-06-25 09:15:30.776519


C:\Users\B042304\AppData\Local\Temp\ipykernel_17372\1331809403.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn_p, chunksize=200000)


2020   11  -  2024-06-25 13:05:03.991314


C:\Users\B042304\AppData\Local\Temp\ipykernel_17372\1331809403.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn_p, chunksize=200000)


2020   12  -  2024-06-25 17:28:04.151777
